In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import sys
sys.path.append('..')
import pytpc
from pytpc.constants import *
from numpy import cos, sin
from scipy.signal import argrelmax, find_peaks_cwt
import skimage.transform as trans

In [ ]:
sns.set_style('white')

In [ ]:
tilt = 7*degrees
clock = 6.25  # MHz
emag = 15e3
bmag = 0#0.5691
ef = np.array([0, 0, emag])
bfield_orig = np.array([0, 0, bmag])
trans = np.array([[1, 0, 0],
                  [0, cos(-tilt), -sin(-tilt)],
                  [0, sin(-tilt), cos(-tilt)]])
bf = np.dot(trans, bfield_orig)
vd = pytpc.simulation.drift_velocity_vector(-2, emag, bmag, tilt)

In [ ]:
# proj = pytpc.Particle(4, 2, 1, position=(0, 0, 1.), azimuth=0, polar=pi+tilt)
proj = pytpc.Particle(4, 2, 2, position=(0, 0, 1.), azimuth=270*degrees, polar=pi - tilt)
target = pytpc.Particle(4, 2, 0)
# heco = pytpc.gases.HeCO2Gas(150.)
heco = pytpc.gases.InterpolatedGasMixture(200., ('helium', 0.9), ('carbon_dioxide', 0.1))

simres = pytpc.simulation.simulate_elastic_scattering_track(proj, target, heco, ef, bf, 4, pi/2, -20*degrees)
# simres = pytpc.simulation.track(proj, heco, ef, bf)
simres.x *= 1000
simres.y *= 1000
simres.z *= 1000
# simres['pos'][:, 2] = 1000 - simres['pos'][:, 2]

In [ ]:
evt = pytpc.evtdata.make_event(simres[['x', 'y', 'z']].values, simres.de.values, clock, 3, 
                               40.2, proj.mass_num, 334., 100)

In [ ]:
pytpc.chamber_plot(evt.xyzs(peaks_only=True));

In [ ]:
pr = pd.DataFrame(evt.xyzs(peaks_only=True)[:, 0:3], columns=('x', 'y', 'z'))

In [ ]:
plt.plot(evt.traces['data'][18])

In [ ]:
# pr.x -= pr.x.mean()
# pr.y -= pr.y.mean()

In [ ]:
pytpc.pad_plot(evt.hits());

In [ ]:
plt.scatter(pr.x, pr.y)
plt.gca().set_aspect('equal')

In [ ]:
plt.scatter(pr.y, pr.z)
plt.gca().set_aspect('equal')

In [ ]:
plt.plot(simres.y, simres.z, '.')
plt.gca().set_aspect('equal')

In [ ]:
plt.plot(simres.z, simres.de, '.')

In [ ]:
plt.plot(evt.traces['data'].sum(0))

In [ ]:
pr['a'] = pr.x / (pr.x**2 + pr.y**2)
pr['b'] = pr.y / (pr.x**2 + pr.y**2)

In [ ]:
plt.scatter(pr.a, pr.b)
plt.gca().set_aspect('equal')
plt.xlim(-0.5, 0.5)
plt.ylim(-0.5, 0.5)

In [ ]:
plt.axes(axisbg='k')
angles = np.linspace(0, pi, 500)
plt.plot(angles*180/pi, [pr.y * cos(th) + pr.z * sin(th) for th in angles], 'w-', linewidth=0.5, alpha=0.5);
# plt.ylim(-0.1, 0.1)
plt.xlim(0, 180)

In [ ]:
gridpts = []
for th in range(180):
    gridpts.append(np.vstack((np.full_like(pr.index, th), pr.y.values * cos(th*pi/180) + pr.z.values * sin(th*pi/180))).T)
gridpts = np.concatenate(gridpts)

In [ ]:
hct, xedge, yedge, *junk = plt.hist2d(gridpts[:, 0], gridpts[:, 1], bins=(180, 600), cmap='Greys_r');
plt.colorbar(junk[-1])
# plt.ylim(-0.1, 0.1)

In [ ]:
plt.vlines(np.arange(len(hct.ravel())), 0, hct.ravel())

In [ ]:
inds, = argrelmax(hct.ravel(), order=5000)

In [ ]:
# inds = np.argpartition(hct.ravel(), -2)[-2:]
xind, yind = np.unravel_index(inds, hct.shape)
th_max, r_max = xedge[xind], yedge[yind]
print(r_max, th_max)

In [ ]:
def invert(x, rho, th):
    return (rho - x*cos(th)) / sin(th)

In [ ]:
def conf_inv(a, b):
    return a/(a**2 + b**2), b/(a**2+b**2)

In [ ]:
curve = pd.DataFrame()
xvals = np.linspace(pr.y.min(), pr.y.max())
curve['a'] = xvals
curve['b'] = [invert(x, r_max[0], th_max[0]*pi/180) for x in xvals]

In [ ]:
curve['x'], curve['y'] = conf_inv(curve.a, curve.b)

In [ ]:
plt.scatter(pr.a, pr.b)
plt.plot(curve.a, curve.b, 'r')

In [ ]:
plt.scatter(pr.x, pr.y)
plt.plot(curve.x, curve.y)
# plt.xlim(pr.x.min(), pr.x.max())
# plt.ylim(pr.y.min(), pr.y.max())

In [ ]:
counts, xedge, yedge, img = plt.hist2d(pr.x, pr.y, bins=(100, 100));

In [ ]:
plt.imshow(counts)

In [ ]:
hspace, angles, dists = trans.hough_circle(counts, 2)
plt.imshow(hspace)
hspace, angles, dists = trans.hough_line_peaks(hspace, angles, dists, threshold=0.4*hspace.max())
# print(len(angles), 'peaks found')

In [ ]:
plt.imshow(counts)
xs = np.linspace(0, 100, 100)
plt.plot(xs, [invert(x, dists, angles) for x in xs])
# plt.scatter(pr.y, pr.z)
plt.xlim(0, counts.shape[0])
plt.ylim(0, counts.shape[1])

# Experimental Data

In [ ]:
ef = pytpc.EventFile('/Users/josh/Documents/Data/Alphas-Dec14/run_0213_ps.evt')

In [ ]:
evt = ef[102]

In [ ]:
pytpc.chamber_plot(evt.xyzs());

In [ ]:
xyzs = evt.xyzs(vd, clock, peaks_only=False)
pts = pd.DataFrame(xyzs[:, 0:3], columns=('x', 'y', 'z'))

In [ ]:
cts, *junk = plt.hist2d(pts.z, pts.y, bins=(200, 200));

In [ ]:
hspace, angles, dists = trans.hough_line(cts)
# plt.imshow(hspace)
hspace, angles, dists = trans.hough_line_peaks(hspace, angles, dists,
                                               min_distance=10, min_angle=2)
print(len(angles), 'lines found')
plt.imshow(cts)
xs = np.linspace(0, cts.shape[0], 100)
plt.plot(xs, [invert(x, dists, angles) for x in xs])
# plt.scatter(pr.y, pr.z)
plt.xlim(0, cts.shape[0])
plt.ylim(0, cts.shape[1])